In [2]:
import glob
from dotenv import load_dotenv
import os
from chroma_utils import PrepareDocumentsFolder,VectorStore
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [3]:
load_dotenv()
DB_NAME = os.getenv("VECTOR_DB_NAME")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")

In [4]:
folders = glob.glob("knowledge-base/*")

In [5]:
prepared_documents = PrepareDocumentsFolder(folders)
chunks = prepared_documents.create_chunks(2000,500)

2025-05-05 19:57:22,629 - utils.tools - INFO - Document types found: products, company, contracts, employees


In [6]:
vectorstore = VectorStore(chunks,DB_NAME)

2025-05-05 19:57:22,653 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-05-05 19:57:22,787 - utils.tools - INFO - Deleted existing collection at vector-db
2025-05-05 19:57:23,745 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 19:57:24,126 - utils.tools - INFO - Vectorstore created with 66 documents
2025-05-05 19:57:24,128 - utils.tools - INFO - Vectorstore 'vector-db' has 1,536 dimensions


In [7]:
messages = [
            (
                "system",
                "You are a helpful assistant for Insurellm HR Staff. Your task is to answer questions about regarding to the company.",
            ),
        ]
llm = ChatOpenAI(temperature=0.7, model_name=OPENAI_MODEL,streaming=True)
llm.invoke(messages)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

2025-05-05 19:57:24,568 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/var/folders/w2/nyq_87_d7l5dzyd9hf6z84lw0000gn/T/ipykernel_88066/2146861124.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [8]:
manual_query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":manual_query})
print(result["answer"])

2025-05-05 19:57:25,483 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 19:57:26,171 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Insurellm is an innovative insurance tech startup founded in 2015 by Avery Lancaster, aimed at disrupting the insurance industry with creative products. The company has grown to 200 employees and operates 12 offices across the US, offering four main software products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With over 300 clients worldwide, Insurellm is focused on providing effective solutions in the insurance technology space.


In [9]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [10]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

2025-05-05 19:57:28,376 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7875/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-05-05 19:57:28,385 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7875/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


2025-05-05 19:57:29,126 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-05-05 19:57:46,283 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 19:57:47,393 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 19:57:48,151 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 19:58:19,513 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 19:58:20,127 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 19:58:20,862 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 19:58:54,082 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 19:58:54,789 - httpx - INFO - HTTP Request